In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [3]:
!kaggle datasets download -d devzohaib/dog-emotions-prediction

100% 379M/379M [00:18<00:00, 23.9MB/s]
100% 379M/379M [00:18<00:00, 22.0MB/s]


In [4]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dog-emotions-prediction.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

# Importing libraries

In [6]:
import numpy as np
import pandas as pd
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dropout, Dense,BatchNormalization, Flatten, MaxPool2D
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from keras.layers import Conv2D, Reshape
from keras.utils import Sequence
from keras.backend import epsilon
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.layers import GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import cv2
import os

In [7]:
print(os.listdir("/content/images/angry"))

['13898046009_1e0a45bbf8_b.jpg', '9711484485_1da72b3519_b.jpg', '1210150680_294ad36b07_b.jpg', '7242538684_52f9e0fb4c_b.jpg', '22268974208_4f56588130_b.jpg', '6814420342_0359d4147d_b.jpg', '10557805643_cbe35a6cc8_b.jpg', '31208953542_ac7823283e_b.jpg', '28146890497_7dc0cb5758_b.jpg', '6950217266_f0f058aecc_b.jpg', '4631867927_b0ea07ef93_b.jpg', '3566935819_fce0a560e7_b.jpg', '3109749372_e4088f5f79_b.jpg', '2785931677_deb9f77d10_b.jpg', '6473647911_71310785ba_b.jpg', '3310197145_ecc49d395f_b.jpg', '8719195532_0f75a45782_b.jpg', '3985554026_395ebce318_b.jpg', '5876554199_4d07253843_b.jpg', '3881296343_ff1436e415_b.jpg', '42067530064_17742a5d30_b.jpg', '1020621584_3895972eda_b.jpg', '248019840_e556a4c349_b.jpg', '2323354248_19a4933dd5_b.jpg', '8127459450_8a36314280_b.jpg', '4722034851_32607b080f_b.jpg', '7926585820_d549400311_b.jpg', '2066663255_a6314aa7bb_b.jpg', '16974030339_4249e828f8_b.jpg', '7705166868_3a0d39ea9e_b.jpg', '7337946614_8734473d18_b.jpg', '856002209_967ccf55ae_b.jpg', '7

# Specifying Paths

In [8]:
angry= '/content/images/angry'
sad = '/content/images/sad/'
relaxed = '/content/images/relaxed/'
happy = '/content/images/happy/'


angry_path = os.listdir(angry)
sad_path = os.listdir(sad)
relaxed_path = os.listdir(relaxed)
happy_path = os.listdir(happy)

In [9]:
data_with_aug = ImageDataGenerator(horizontal_flip=True,
                                   vertical_flip=False,
                                   rescale=1./255,
                                  validation_split=0.3)

## Angry Dogs

In [10]:
import cv2

# Load the image
img = cv2.imread('/content/images/angry/100035066_bb2792d5c8_b.jpg')

# Print the size of the image
print(img.shape)

(384, 384, 3)


In [11]:
dataset_path = "/content/images"

In [12]:
data_with_aug = ImageDataGenerator(horizontal_flip=True,
                                   vertical_flip=False,
                                   rescale=1./255,
                                  validation_split=0.3)

In [13]:
train = data_with_aug.flow_from_directory(dataset_path,
                                          class_mode="binary",
                                          target_size=(96, 96),
                                          batch_size=32,
                                          subset="training")

Found 11147 images belonging to 4 classes.


In [14]:
val = data_with_aug.flow_from_directory(dataset_path,
                                          class_mode="binary",
                                          target_size=(96, 96),
                                          batch_size=32,
                                          subset="validation"
                                          )

Found 4774 images belonging to 4 classes.


# VGG19 Model

In [15]:
train = data_with_aug.flow_from_directory(dataset_path,
                                          class_mode="binary",
                                          target_size=(224, 224),
                                          batch_size=98,
                                          subset="training")

val = data_with_aug.flow_from_directory(dataset_path,
                                          class_mode="binary",
                                          target_size=(224, 224),
                                          batch_size=98,
                                          subset="validation"
                                          )

Found 11147 images belonging to 4 classes.
Found 4774 images belonging to 4 classes.


In [16]:
vgg19_model = tf.keras.applications.vgg19.VGG19(include_top=False, weights="imagenet", input_shape=(224,224,3))

80134624/80134624 [==============================] - 4s 0us/step


In [17]:
vgg19_model.output[-1]

<KerasTensor: shape=(7, 7, 512) dtype=float32 (created by layer 'tf.__operators__.getitem')>

In [18]:
vgg19_model.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [19]:
model = Sequential([vgg19_model,Flatten(),Dense(4, activation = "sigmoid")])
model.layers[0].trainable = False
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics="accuracy")

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 4)                 100356    
                                                                 
Total params: 20,124,740
Trainable params: 100,356
Non-trainable params: 20,024,384
_________________________________________________________________


In [22]:
!pip install pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
hist =  model.fit_generator(train,
                    epochs=20,
                    callbacks=[lr_callbacks],
                    validation_data=val)


<ipython-input-30-598b8c2f3cde>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist =  model.fit_generator(train,


Epoch 1/20
114/114 [==============================] - 89s 784ms/step - loss: 0.8162 - accuracy: 0.6779 - val_loss: 1.3852 - val_accuracy: 0.4537 - lr: 0.0010
Epoch 2/20
114/114 [==============================] - 90s 785ms/step - loss: 0.7919 - accuracy: 0.6871 - val_loss: 1.3927 - val_accuracy: 0.4437 - lr: 0.0010
Epoch 3/20
114/114 [==============================] - 89s 778ms/step - loss: 0.7479 - accuracy: 0.7105 - val_loss: 1.5118 - val_accuracy: 0.4386 - lr: 0.0010
Epoch 4/20
114/114 [==============================] - 90s 787ms/step - loss: 0.7592 - accuracy: 0.6971 - val_loss: 1.6509 - val_accuracy: 0.4009 - lr: 0.0010
Epoch 5/20
114/114 [==============================] - 89s 781ms/step - loss: 0.7222 - accuracy: 0.7181 - val_loss: 1.4485 - val_accuracy: 0.4420 - lr: 0.0010
Epoch 6/20
114/114 [==============================] - 89s 778ms/step - loss: 0.6993 - accuracy: 0.7271 - val_loss: 1.5178 - val_accuracy: 0.4326 - lr: 0.0010
Epoch 7/20
114/114 [==============================] 

In [31]:
predictions = model.predict_generator(val)

<ipython-input-31-7c68dd12cc9d>:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = model.predict_generator(val)
